In [ ]:
def find_anomaly_in_dataset(dirty_path, clean_folder_path):
    print(f"--- ANALIZA OBRAZU: {os.path.basename(dirty_path)} ---")

    # ID bazy badanego pojazdu (np. "48001F003202511180021")
    dirty_base = get_base_id(dirty_path)

    # A. Przygotowanie obrazu BRUDNEGO
    img_dirty_crop = crop_robust_count(dirty_path)
    if img_dirty_crop is None:
        print("Błąd: Nie udało się wykryć auta na zdjęciu badanym.")
        return

    # Miniaturka do szybkiego liczenia MSE
    search_size = (128, 128)
    img_dirty_thumb = cv2.resize(img_dirty_crop, search_size).astype("float32")

    # B. PĘTLA WYSZUKIWANIA (szukamy bliźniaka w folderze czystych)
    clean_files = [f for f in os.listdir(clean_folder_path)
                   if f.lower().endswith('.bmp')]

    if not clean_files:
        print("Błąd: Pusty folder ze zdjęciami wzorcowymi.")
        return

    # od razu odfiltrowujemy wszystkie pliki tej samej bazy (czarne / kolor)
    candidate_files = [f for f in clean_files
                       if get_base_id(f) != dirty_base]

    if not candidate_files:
        print("Brak innych pojazdów w bazie (po wykluczeniu tej samej bazy).")
        return

    best_score = float('inf')
    best_clean_crop = None
    best_filename = ""

    print(f"Przeszukiwanie bazy {len(candidate_files)} zdjęć wzorcowych...")

    for f in candidate_files:
        path = os.path.join(clean_folder_path, f)

        # 1. Wycinamy auto ze zdjęcia czystego
        clean_crop = crop_robust_count(path)
        if clean_crop is None:
            continue

        # 2. Skalujemy wycinek do miniaturki
        clean_thumb = cv2.resize(clean_crop, search_size).astype("float32")

        # 3. Porównujemy (MSE - błąd średniokwadratowy)
        score = np.mean((img_dirty_thumb - clean_thumb) ** 2)

        if score < best_score:
            best_score = score
            best_clean_crop = clean_crop   # zapisujemy sobie pełny, wycięty obraz
            best_filename = f

    if best_clean_crop is None:
        print("Nie znaleziono pasującego obrazu wzorcowego.")
        return

    print(f"ZNALEZIONO BLIŹNIAKA: {best_filename} (Błąd dopasowania: {best_score:.2f})")

    # C. PROCESOWANIE RÓŻNIC (logika wykrywania anomalii)

    # 1. Dopasowanie wymiarów
    h_target, w_target = img_dirty_crop.shape
    img_clean_resized = cv2.resize(best_clean_crop, (w_target, h_target))

    # 2. Rozmycie (usuwanie ziarna matrycy)
    k_blur = 5
    dirty_blur = cv2.GaussianBlur(img_dirty_crop, (k_blur, k_blur), 0)
    clean_blur = cv2.GaussianBlur(img_clean_resized, (k_blur, k_blur), 0)

    # 3. Odejmowanie (wartość bezwzględna)
    diff = cv2.absdiff(dirty_blur, clean_blur)

    # 4. Progowanie (threshold)
    _, mask = cv2.threshold(diff, 45, 255, cv2.THRESH_BINARY)

    # 5. EROZJA (usuwanie "duchów")
    kernel = np.ones((3, 3), np.uint8)
    mask_cleaned = cv2.erode(mask, kernel, iterations=2)

    # 6. DYLATACJA (przywracanie kształtu)
    mask_final = cv2.dilate(mask_cleaned, kernel, iterations=4)

    # 7. ANALIZA SKŁADOWYCH POŁĄCZONYCH (filtr na blobach)
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(
        mask_final, connectivity=8
    )

    h, w = mask_final.shape
    filtered = np.zeros_like(mask_final)

    # progi do strojenia
    MIN_AREA = 200          # za małe -> szum
    MAX_AREA = 15000        # za duże -> raczej przesunięcie / tło
    MIN_FILL = 0.15         # jak bardzo prostokąt jest "wypełniony"
    MIN_AR, MAX_AR = 0.4, 4.0   # dopuszczalny stosunek szer/szer
    BORDER_MARGIN = 5           # ile pikseli od brzegu ignorujemy

    for label in range(1, num_labels):  # 0 = tło
        x, y, w_box, h_box, area = stats[label]

        # 1) filtr po powierzchni
        if area < MIN_AREA or area > MAX_AREA:
            continue

        # 2) filtr po proporcjach (obcinamy bardzo cienkie linie)
        aspect = w_box / float(h_box + 1e-6)
        if not (MIN_AR <= aspect <= MAX_AR):
            continue

        # 3) jak bardzo wypełniony jest prostokąt (obcinamy "kratki" z kół itd.)
        rect_area = w_box * h_box
        fill = area / float(rect_area + 1e-6)
        if fill < MIN_FILL:
            continue

        # 4) ignorujemy obiekty dotykające ramki (artefakt dopasowania)
        if (x <= BORDER_MARGIN or y <= BORDER_MARGIN or
            x + w_box >= w - BORDER_MARGIN or
            y + h_box >= h - BORDER_MARGIN):
            continue

        # jeśli przeszło wszystkie filtry – wrzucamy do wyjściowej maski
        filtered[labels == label] = 255

    mask_final_filtered = filtered

    # --- D. WIZUALIZACJA ---
    plt.figure(figsize=(16, 10))

    plt.subplot(2, 2, 1)
    plt.title("1. Badany (Wycinek)")
    plt.imshow(img_dirty_crop, cmap='gray')
    plt.axis('off')

    plt.subplot(2, 2, 2)
    plt.title(f"2. Wzorzec (Dopasowany): {best_filename}")
    plt.imshow(img_clean_resized, cmap='gray')
    plt.axis('off')

    plt.subplot(2, 2, 3)
    plt.title("3. Różnica surowa (widać szumy i przesunięcia)")
    plt.imshow(diff, cmap='gray')
    plt.axis('off')

    plt.subplot(2, 2, 4)
    plt.title("4. WYKRYTE ANOMALIE (Po filtracji)")
    plt.imshow(mask_final_filtered, cmap='gray')
    plt.axis('off')

    plt.tight_layout()
    plt.show()